# Meshing a Cylinder with Gmsh

## Mesh the cylinder with all points and cells

In [ ]:
import gmsh
import warnings
import numpy as np

warnings.filterwarnings("ignore")
gmsh.initialize() # initializing the gmsh interface

# parameters for a cylinder
x0=[0.0,0.0,0.0] # initial point of the cylinder (x,y,z coordinates)
axis=[50.8,0.0,0.0] # extension of the cylinder aka change in the x,y,z direction (dx,dy,dz)
radius=12.7 # radius of he cylinder
max_char_len=2.0 # maximum characteristic length of the elements

# add the cylinder
gmsh.model.occ.add_cylinder(x=x0[0],y=x0[1],z=x0[2],dx=axis[0],dy=axis[1],dz=axis[2],r=radius)

all_points = gmsh.model.occ.getEntities(0) # obtain all the points in the mesh
gmsh.model.occ.mesh.setSize(dimTags=all_points,size=max_char_len) # set the characteristic lengh of the mesh
gmsh.model.occ.synchronize() # internal gmsh code to make the meshing works
gmsh.model.mesh.generate(3) # generate the mesh in 3D


gmsh.write("mesh/cylinder_long.msh") # save the mesh
gmsh.finalize() # close gmsh

## Mesh the cylinder facets and label the facets (to apply boundary conditions later)

In [ ]:
import gmsh
import warnings
import numpy as np

warnings.filterwarnings("ignore")
gmsh.initialize() # initializing the gmsh interface

# parameters for a cylinder
x0=[0.0,0.0,0.0] # initial point of the cylinder (x,y,z coordinates)
axis=[50.8,0.0,0.0] # extension of the cylinder aka change in the x,y,z direction (dx,dy,dz)
radius=12.7 # radius of he cylinder
max_char_len=2.0 # maximum characteristic length of the elements

# add the cylinder
gmsh.model.occ.add_cylinder(x=x0[0],y=x0[1],z=x0[2],dx=axis[0],dy=axis[1],dz=axis[2],r=radius)

all_points = gmsh.model.occ.getEntities(0) # obtain all the points in the mesh
surfaces = gmsh.model.occ.getEntities(dim=2) # obtain all the facet entities
gmsh.model.occ.mesh.setSize(dimTags=all_points,size=max_char_len) # set the characteristic lengh of the mesh
gmsh.model.occ.synchronize() # internal gmsh code to make the meshing works

side_marker, top_marker, bot_marker = 7,101,102 # determine facet indexing
walls = []
obstacles = []
for surface in surfaces:
    com = gmsh.model.occ.getCenterOfMass(surface[0], surface[1])

    # if the x position of the facet is near 50.8, assign top index to facet
    if np.allclose(com[0], 50.8): 
        gmsh.model.addPhysicalGroup(surface[0], [surface[1]], top_marker) 
    # if the x position of the facet is near 0.0, assign bot index to facet
    elif np.allclose(com[0], 0):
        gmsh.model.addPhysicalGroup(surface[0], [surface[1]], bot_marker)
    # else, assign side index to facet
    else:
        gmsh.model.addPhysicalGroup(surface[0], [surface[1]], side_marker)

gmsh.model.mesh.generate(3) # generate mesh in 3D

gmsh.write("mesh/facet_cylinder_long.msh") # save mesh
gmsh.finalize() # close gmsh

## Convert .msh file type to .xdmf file type for compatibility with FEniCSx

In [ ]:

"""
Created on Fri Aug 11 16:37:26 2023

@author: eric
"""

# Change the name of the gmsh file below  stored in the folder meshes from 3d_hip_v2.msh
# to the file  *.gmsh that you wish to convert. Accordingly, also change the names 
# of the output files  facet_3D_hip_v2.xdmf and  3D_hip_v2.xdmf.

"""
Step 1. convert the mesh to.xdmf  format using meshio
"""
import meshio
mesh_from_file = meshio.read("mesh/cylinder_long.msh")
facet_from_file = meshio.read("mesh/facet_cylinder_long.msh")


"""
Step. 2  Extract cells and boundary data.

Now that we have created the mesh, we need to extract the cells 
and physical data. We need to create a separate file for the 
facets (lines),  which we will use when we define boundary 
conditions in  Fenics. We do this  with the following convenience 
function. Note that as we would like a  2 dimensional mesh, we need to 
remove the z-values in the mesh coordinates, if any.
"""

def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
    if cell_type=="triangle":
        cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    elif cell_type=="tetra":
        cell_data = mesh.get_cell_data("gmsh:geometrical", cell_type)
    points = mesh.points[:,:2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells},\
                           cell_data={"name_to_read":[cell_data]})
    return out_mesh

"""
Step 3.
With this function in hand, we can save the facet line mesh 
and the domain triangle  mesh in `XDMF` format 
"""

triangle_mesh = create_mesh(facet_from_file, "triangle", prune_z=False)
meshio.write("mesh/facet_cylinder_long.xdmf", triangle_mesh)

tetra_mesh = create_mesh(mesh_from_file, "tetra", prune_z=False)
meshio.write("mesh/cylinder_long.xdmf", tetra_mesh)